In [1]:
# from cifar10tools import test, make_model, VGG, make_layers, calculate_params
from svhn_tools import test, make_model, VGG, make_layers, calculate_params
from torchvision import datasets
from torchvision.transforms import transforms
from torch.utils.data import Subset, DataLoader

import pickle
import torch
import numpy as np

In [2]:
class Individual():
    def __init__(self, bitstring) -> None:
        self.bitstring = bitstring
        self.metrics = None
    
    def mutate(self):
        mp_list = [0.0002]
        mutation_prob = mp_list[np.random.randint(len(mp_list))]
        self.metrics = None
        self

cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',
          512, 512, 512, 512, 'M'],
}

# base_model = VGG(make_layers(
#     cfg['D'] ## vgg1g
# ))
base_model = VGG(make_layers(
    cfg['E'] ## vgg19
))
# base_model.load_state_dict(torch.load('models/vgg16_cifar10_base_model'))
base_model.load_state_dict(torch.load('models/vgg19_svhn_base_model'))
base_model = base_model.to('cuda')

## Test the last saved model
population = pickle.load(open('./checkpoints/pruned_population_ckpt/last_svhn_gen.p','rb'))
# population = pickle.load(open('./checkpoints/pruned_population_ckpt/last_gen_best_vgg16.p','rb'))
best_indiv = max(population, key=lambda x: x.metrics['score'])

pruned_model = make_model(best_indiv.bitstring, base_model)

# dataset_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ]))

dataset_test = datasets.SVHN(root='./data', split='test', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))

dataloader = DataLoader(
    dataset_test,
    batch_size=2048, shuffle=True,
    num_workers=4, pin_memory=True)
pruned_acc, base_acc = test(dataloader, [pruned_model, base_model])
print(pruned_acc, base_acc)
print(f'ACC DROP:{(base_acc-pruned_acc)*100/base_acc}%')

Using downloaded and verified file: ./data/test_32x32.mat


ModuleNotFoundError: No module named 'scipy'

Parameter drop of the pruned model

In [ ]:
base_params_count = calculate_params(base_model)
pruned_params_count = calculate_params(pruned_model)
print(f'BASE PRAMS:{base_params_count}')
print(f'PRUNED PARAMS:{pruned_params_count}')
print(f'PARAMS DROP:{(base_params_count - pruned_params_count)*100 / base_params_count}%')

BASE PRAMS:20554826
PRUNED PARAMS:19199962
PARAMS DROP:6.591464213805556%


Calculate FLOPS of pruned and base model

In [ ]:
from fvcore.nn import FlopCountAnalysis

random_input = dataset_test[np.random.randint(len(dataset_test))][0]
random_input = random_input.unsqueeze(0)
random_input = random_input.to('cuda')

base_flops_analysis = FlopCountAnalysis(base_model, random_input)
pruned_flops_analysis = FlopCountAnalysis(pruned_model, random_input)

base_flops = base_flops_analysis.total()
pruned_flops = pruned_flops_analysis.total()

print('BASE   FLOPS:',base_flops)
print('PRUNED FLOPS:',pruned_flops)

print('FLOP DROP:', f'{(base_flops - pruned_flops)*100/base_flops}%')

Unsupported operator aten::max_pool2d encountered 5 time(s)
Unsupported operator aten::max_pool2d encountered 5 time(s)


BASE   FLOPS: 398660608
PRUNED FLOPS: 379488084
FLOP DROP: 4.809234625960336%


In [ ]:
## Compression ratio
CR = (base_params_count/pruned_params_count)
print(CR)

1.0705659729951549
